# CoreNLP

In [1]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza

In [2]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
# stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

In [3]:
# Import client module
from stanza.server import CoreNLPClient

In [4]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=["coref"], 
    memory='4G', 
    endpoint='http://localhost:9001',
    be_quiet=True)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()

2021-01-12 21:47:23 INFO: Writing properties to tmp file: corenlp_server-e1d6675da6404693.props
2021-01-12 21:47:23 INFO: Starting server with command: java -Xmx4G -cp ./corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-e1d6675da6404693.props -annotators coref -preload -outputFormat serialized


In [7]:
# Annotate some text
text = "I really recommend this phone. Great battery life, good camera, 90 fps display, stock android, large display but not bulky at the same time, comes in 5 colours and 2 finishes. It would have been better though if we had an option to upgrade the RAM. It becomes laggy after a few days unless you recharge."

In [10]:
document = client.annotate(text)

In [11]:
mychains = list()
chains = document.corefChain
for chain in chains:
    mychain = list()
    # Loop through every mention of this chain
    for mention in chain.mention:
        # Get the sentence in which this mention is located, and get the words which are part of this mention
        # (we can have more than one word, for example, a mention can be a pronoun like "he", but also a compound noun like "His wife Michelle")
        words_list = document.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
        #build a string out of the words of this mention
        ment_word = ' '.join([x.word for x in words_list])
        mychain.append(ment_word)
    mychains.append(mychain)

for chain in mychains:
    print(' <-> '.join(chain))

It <-> It


In [12]:
client.stop()

### Reference: 
1. https://www.rangakrish.com/index.php/2019/02/10/coreference-resolution-in-stanford-corenlp/
2. https://github.com/aleenaraj/Coreference_Resolution/blob/master/CoreNLP_coref.ipynb

# NeuralCoref

In [10]:
# !pip install spacy==2.1.0  # version 2.1.0 is only compatible with neuralcoref
# !pip install neuralcoref --no-binary neuralcoref

In [26]:
import spacy
import neuralcoref
nlp = spacy.load('en_core_web_md')
neuralcoref.add_to_pipe(nlp, greedyness=0.6)
# Annotate some text
# text = "I bought this light. It works good but battery is not. It discharges quickly."
doc = nlp(text)
print(doc._.coref_clusters)

[this phone: [this phone, Great battery life, good camera, 90 fps display, stock android, large display but not bulky at the same time, 90 fps display, stock android, large display but not bulky, stock android, large display but not bulky, large display but not bulky, the same time, 5 colours and 2 finishes, 2 finishes, It, we, we had an option to upgrade the RAM, an option to upgrade the RAM, the RAM, RAM, It, a few days, you recharge]]


### References:
1. https://medium.com/huggingface/state-of-the-art-neural-coreference-resolution-for-chatbots-3302365dcf30
2. https://github.com/huggingface/neuralcoref
3. https://stackoverflow.com/questions/63668616/en-coref-lg-model-in-spacy
4. https://www.rangakrish.com/index.php/2019/02/03/coreference-resolution-using-spacy/